# Introduction to SQL

## Introduction
#### Goals

   1. Explore the Database
   2. Basic Querying - Selecting From Tables
   3. Selecting specific attributes of a table
   4. Where clause/ filtering
   5. Aggregation functions: counting
   6. Aggregation functions: AVG
   7. Intervals, Ranges, and sorting
   8. Subqueries


## Basic
### Part 1: Loading the database

In this repo, there's a SQL dump of the data we'll be using today. This data is typical of the type of data you might encounter in industry. It is [normalized](https://en.wikipedia.org/wiki/Database_normalization), which is a way of minimizing the disk space required to store the data and ensuring consistency, but which can sometimes require more effort to get data, since most queries will require information stored across multiple tables. As an example, the events table has ids for both users and meals, but in order to get the price of the meal, we have to look up that meal in the id table. This allows us to use _only_ the id to refer to the meal _anywhere_ it may appear, but does mean that to get meal details we almost always have to join. *As a data scientist, you will be writing a lot of SQL in order to get data from various tables into a single location where you can use it.*

1. If you are on your personal computer and haven't set up postgres yet, follow [these instructions](https://github.com/GalvanizeDataScience/docker/blob/master/reference/docker_postgres.md) to start a docker container.

1. **From a Docker bash shell** run `psql -U postgres` and then this command to create the database. See [here for more details on loading the data](https://github.com/GalvanizeDataScience/docker/blob/master/guides/docker_postgres.md)

    ```sql
    CREATE DATABASE readychef;
    \q
    ```

1. **Back in the Docker bash shell** navigate to where the data on your mounted folder is:

    ```
    cd data
    psql -U postgres readychef < readychef.sql
    ```

    You should see a bunch of SQL commands flow through the terminal. 

1. To enter the interactive Postgres command-line client and connect to your database, enter the following from the bash shell in the docker container.

    ```
    psql -U postgres readychef
    ```
This will put you in the command-line client. We won't use that for this assignment, but you can type

    ```sql
    SELECT * FROM users LIMIT 10; 
    ```

to see the first rows of the users table and verify the database was set up correctly (don't forget the semicolon!). We can do
    ```
    \d
    ```
in the command-line client to list all the tables in the database, and look at details of the user table with
    ```
    \d users
    ```

When you've looked at the details for each table, you can enter
    ```
    \q
    ```

to exit the command-line client, but you might keep this open to refer to it again.


### Part 2: Basic Exploration

For this assignment we'll access the postgres database from python using a jupyter notebook.

1. Import `psycopg2` and create a connection to the database with

    ```python
    conn = psycopg2.connect(dbname='readychef',
                        host='localhost',
                        user='postgres',
                        password='password')
    ```

In [2]:
import psycopg2
import pandas as pd

In [4]:
conn = psycopg2.connect(dbname='readychef',
                        host='localhost',
                        user='postgres',
                        password='galvanize')

In [5]:
cur = conn.cursor()
cur.execute("""SELECT * FROM events LIMIT 10;""")
for row in cur:
    print(row)

(datetime.date(2013, 1, 1), 3, 18, 'bought')
(datetime.date(2013, 1, 1), 7, 1, 'like')
(datetime.date(2013, 1, 1), 10, 29, 'bought')
(datetime.date(2013, 1, 1), 11, 19, 'share')
(datetime.date(2013, 1, 1), 15, 33, 'like')
(datetime.date(2013, 1, 1), 18, 4, 'share')
(datetime.date(2013, 1, 1), 18, 40, 'bought')
(datetime.date(2013, 1, 1), 21, 10, 'share')
(datetime.date(2013, 1, 1), 21, 4, 'like')
(datetime.date(2013, 1, 1), 22, 23, 'bought')


3. Write a function that takes a query string and database connection, executes the query, and returns a list of rows. Call the function on a statement selecting the first 10 rows of the users table. Is it better for a function to return a list of rows or the cursor itself?

> Regarding which is better: it depends which is better. Loading it into a list mean that other functions that call it won't need to manipulate the cursor. But loading it into a list puts the entire result in memory, when the calling function might only need a line at a time.

In [5]:
def execute_query(query, conn):
    cur = conn.cursor()
    cur.execute(query)
    return list(cur)

In [6]:
execute_query("""SELECT * FROM events LIMIT 10;""", conn)

[(datetime.date(2013, 1, 1), 3, 18, 'bought'),
 (datetime.date(2013, 1, 1), 7, 1, 'like'),
 (datetime.date(2013, 1, 1), 10, 29, 'bought'),
 (datetime.date(2013, 1, 1), 11, 19, 'share'),
 (datetime.date(2013, 1, 1), 15, 33, 'like'),
 (datetime.date(2013, 1, 1), 18, 4, 'share'),
 (datetime.date(2013, 1, 1), 18, 40, 'bought'),
 (datetime.date(2013, 1, 1), 21, 10, 'share'),
 (datetime.date(2013, 1, 1), 21, 4, 'like'),
 (datetime.date(2013, 1, 1), 22, 23, 'bought')]

In [7]:
pd.read_sql("""SELECT * FROM events LIMIT 10;""", conn)

,dt,userid,meal_id,event
0,2013-01-01,3,18,bought
1,2013-01-01,7,1,like
2,2013-01-01,10,29,bought
3,2013-01-01,11,19,share
4,2013-01-01,15,33,like
5,2013-01-01,18,4,share
6,2013-01-01,18,40,bought
7,2013-01-01,21,10,share
8,2013-01-01,21,4,like
9,2013-01-01,22,23,bought


### Part 3: Select statements

1. To get an understanding of the data, run a [SELECT](http://www.postgresqltutorial.com/postgresql-select/) statement on each table. Keep all the columns and limit the number of rows to 10.

In [8]:
pd.read_sql("""SELECT * FROM events LIMIT 10;""", conn)

,dt,userid,meal_id,event
0,2013-01-01,3,18,bought
1,2013-01-01,7,1,like
2,2013-01-01,10,29,bought
3,2013-01-01,11,19,share
4,2013-01-01,15,33,like
5,2013-01-01,18,4,share
6,2013-01-01,18,40,bought
7,2013-01-01,21,10,share
8,2013-01-01,21,4,like
9,2013-01-01,22,23,bought


In [9]:
pd.read_sql("""SELECT * FROM meals LIMIT 10;""", conn)

,meal_id,type,dt,price
0,1,french,2013-01-01,10
1,2,chinese,2013-01-01,13
2,3,mexican,2013-01-02,9
3,4,italian,2013-01-03,9
4,5,chinese,2013-01-03,12
5,6,italian,2013-01-03,9
6,7,italian,2013-01-03,10
7,8,french,2013-01-03,14
8,9,italian,2013-01-03,13
9,10,french,2013-01-03,7


In [10]:
pd.read_sql("""SELECT * FROM referrals LIMIT 10;""", conn)

,referred,referred_by
0,54,44
1,158,80
2,184,139
3,263,107
4,275,35
5,279,6
6,298,167
7,305,59
8,311,80
9,330,289


In [11]:
pd.read_sql("""SELECT * FROM users LIMIT 10;""", conn)

,userid,dt,campaign_id
0,1,2013-01-01,RE
1,2,2013-01-01,PI
2,3,2013-01-01,FB
3,4,2013-01-01,FB
4,5,2013-01-01,FB
5,6,2013-01-01,FB
6,7,2013-01-01,PI
7,8,2013-01-01,FB
8,9,2013-01-01,FB
9,10,2013-01-01,TW


In [12]:
pd.read_sql("""SELECT * FROM visits LIMIT 10;""", conn)

,dt,userid
0,2013-01-01,3
1,2013-01-01,7
2,2013-01-01,8
3,2013-01-01,10
4,2013-01-01,11
5,2013-01-01,15
6,2013-01-01,18
7,2013-01-01,19
8,2013-01-01,20
9,2013-01-01,21


2. Write a `SELECT` statement that would get just the userids from the events table.

In [13]:
pd.read_sql("""SELECT userid FROM events LIMIT 10;""", conn)

,userid
0,3
1,7
2,10
3,11
4,15
5,18
6,18
7,21
8,21
9,22


3. Maybe you're just interested in what the campaign ids are. Use 'SELECT DISTINCT' to figure out all the possible values of that column in the appropriate table.

    *Note:*  Pinterest=PI, Facebook=FB, Twitter=TW, and Reddit=RE

In [14]:
pd.read_sql("""SELECT DISTINCT campaign_id FROM users;""",
            conn)

,campaign_id
0,FB
1,RE
2,PI
3,TW


### Part 4: Where Clauses / Filtering

Now that we have the lay of the land, we're interested in the subset of users that came from Facebook (FB). If you're unfamiliar with SQL syntax, the [WHERE](http://www.postgresqltutorial.com/postgresql-where/) clause can be used to add a conditional to `SELECT` statements. This has the effect of only returning rows where the conditional evaluates to `TRUE`. 

*Note: Make sure you put string literals in single quotes, like `campaign_id='TW'`.*

1. Using the `WHERE` clause, write a new `SELECT` statement that returns all rows where `Campaign_ID` is equal to `FB`.



In [15]:
pd.read_sql("""SELECT * from users
            where campaign_id = 'FB';""",
            conn)

,userid,dt,campaign_id
0,3,2013-01-01,FB
1,4,2013-01-01,FB
2,5,2013-01-01,FB
3,6,2013-01-01,FB
4,8,2013-01-01,FB
...,...,...,...
2187,5511,2013-12-31,FB
2188,5513,2013-12-31,FB
2189,5516,2013-12-31,FB
2190,5518,2013-12-31,FB


2. We don't need the campaign id in the result since they are all the same, so only include the other two columns.

    Your output should be something like this:

    ```
userid	dt
0	3	2013-01-01
1	4	2013-01-01
2	5	2013-01-01
3	6	2013-01-01
4	8	2013-01-01
...
    ```

### Part 5: Aggregation Functions

Let's try some [aggregation functions](http://www.postgresql.org/docs/8.2/static/functions-aggregate.html) now.

`COUNT` is an example aggregate function, which counts all the entries and you can use like this:

```sql
SELECT COUNT(*) FROM users;
```

Your output should look something like:

```
    count
0	5524

```

1. Write a query to get the count of just the users who came from Facebook.

In [16]:
pd.read_sql("""SELECT COUNT(*) FROM users
            WHERE campaign_id = 'FB';""",
            conn)

,count
0,2192


2. Now, count the number of users coming from each service. Here you'll have to group by the column you're selecting with a [GROUP BY](http://www.postgresql.org/docs/8.0/static/sql-select.html#SQL-GROUPBY) clause.

In [17]:
pd.read_sql("""SELECT campaign_id, COUNT(*) FROM users
            GROUP BY campaign_id;""",
            conn)

,campaign_id,count
0,FB,2192
1,RE,862
2,PI,588
3,TW,1882


    Try running the query without a group by. Postgres will tell you what to put in your group by clause!

In [18]:
pd.read_sql("""SELECT campaign_id, COUNT(*) FROM users;""",
            conn)

DatabaseError: Execution failed on sql 'SELECT campaign_id, COUNT(*) FROM users;': column "users.campaign_id" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: SELECT campaign_id, COUNT(*) FROM users;
               ^


> Since we had an error, we'll need to re-create our connection.

In [39]:
conn = psycopg2.connect(dbname='readychef',
                        host='localhost',
                        user='postgres',
                        password='galvanize', port = 8997)

3. Use `COUNT (DISTINCT columnname)` to get the number of unique dates that appear in the `users` table.

In [40]:
pd.read_sql("""SELECT COUNT(DISTINCT(dt)) FROM users;""",
            conn)

,count
0,352


4. There's also `MAX` and `MIN` functions, which do what you might expect. Write a query to get the first and last registration date from the `users` table.

In [21]:
pd.read_sql("""SELECT MIN(dt), MAX(dt)
            FROM users;""",
            conn)

,min,max
0,2013-01-01,2013-12-31


5. Calculate the mean price for a meal (from the `meals` table). You can use the `AVG` function. Your result should look like this:

    ```
	avg
0	10.652283
    ```

In [22]:
pd.read_sql("""SELECT AVG(price)
            FROM meals;""",
            conn)

,avg
0,10.652283


6. Now get the average price, the min price and the max price for each meal type. Don't forget the group by statement!

    Your output should look like this:

    ```
	type	avg	min	max
0	mexican	9.697595	6	13
1	italian	11.292614	7	16
2	chinese	9.518717	6	13
3	french	11.542000	7	16
4	japanese	9.380488	6	13
5	vietnamese	9.283019	6	13
    ```

7. It's often helpful for us to give our own names to columns. We can always rename columns that we select by doing `AVG(price) AS avg_price`. This is called [aliasing](http://stackoverflow.com/questions/15413735/postgresql-help-me-figure-out-how-to-use-table-aliases). Alias all the above columns so that your table looks like this:

    ```
type	avg_price	min_price	max_price
0	mexican	9.697595	6	13
    ...
    ```

In [23]:
pd.read_sql("""SELECT type,
                      AVG(price) as avg_price,
                      MIN(price) as min_price,
                      MAX(price) as max_price
               FROM meals
               GROUP BY type;""",
            conn)

,type,avg_price,min_price,max_price
0,mexican,9.697595,6,13
1,italian,11.292614,7,16
2,chinese,9.518717,6,13
3,french,11.542000,7,16
4,japanese,9.380488,6,13
5,vietnamese,9.283019,6,13


8. Maybe you only want to consider the meals which occur in the first quarter (January through March). Use `date_part` to get the month like this: `date_part('month', dt)`. Add a `WHERE` clause to the above query to consider only meals in the first quarter of 2013 (month<=3 and year=2013).

In [24]:
pd.read_sql("""SELECT type,
                      AVG(price) as avg_price,
                      MIN(price) as min_price,
                      MAX(price) as max_price
               FROM meals
               WHERE DATE_PART('month', dt) <= 3 AND
                     DATE_PART('year', dt) = 2013
               GROUP BY type;""",
            conn)

,type,avg_price,min_price,max_price
0,chinese,9.772727,6,13
1,french,11.752212,7,16
2,italian,11.087719,7,16
3,japanese,9.652174,6,13
4,mexican,9.695122,6,13
5,vietnamese,9.375000,6,13


9. There are also scenarios where you'd want to group by two columns. Modify the above query so that we get the aggregate values for each month and type. You'll need to add the month to both the select statement and the group by statement.

    It'll be helpful to *alias* the month column and give it a name like `month` so you don't have to call the `date_time` function again in the `GROUP BY` clause.

    Your result should look like this:
```
       type	month	avg_price	min_price	max_price
0	chinese	1.0	11.230769	8	13
1	chinese	2.0	9.066667	6	13
2	chinese	3.0	9.250000	6	13
3	french	1.0	11.650000	7	16
```

In [25]:
pd.read_sql("""SELECT type,
                      DATE_PART('month', dt) as month,
                      AVG(price) as avg_price,
                      MIN(price) as min_price,
                      MAX(price) as max_price
               FROM meals
               WHERE DATE_PART('month', dt) <= 3 AND
                     DATE_PART('year', dt) = 2013
               GROUP BY type, DATE_PART('month', dt);""",
            conn)

,type,month,avg_price,min_price,max_price
0,chinese,1.0,11.230769,8,13
1,chinese,2.0,9.066667,6,13
2,chinese,3.0,9.250000,6,13
3,french,1.0,11.650000,7,16
4,french,2.0,10.838710,7,16
5,french,3.0,12.523810,8,16
6,italian,1.0,10.803030,7,16
7,italian,2.0,11.266667,7,16
8,italian,3.0,11.266667,7,16
9,japanese,1.0,9.615385,6,13


10. From the `events` table, write a query that gets the total number of buys, likes and shares for each meal id. 

_Extra_: To avoid having to do this as three separate queries you can do the count of the number of buys like this: `SUM(CASE WHEN event='bought' THEN 1 ELSE 0 END)`.

In [26]:
pd.read_sql("""SELECT meal_id,
                      SUM(CASE WHEN event='bought' THEN 1 ELSE 0 END) as buys, 
                      SUM(CASE WHEN event='like' THEN 1 ELSE 0 END) as likes,
                      SUM(CASE WHEN event='share' THEN 1 ELSE 0 END) as shares
               FROM events
               GROUP BY meal_id;""",
            conn)

,meal_id,buys,likes,shares
0,1798,56,105,122
1,1489,67,107,133
2,1269,47,90,112
3,652,16,41,42
4,273,6,13,14
...,...,...,...,...
1981,520,22,37,47
1982,1715,52,95,109
1983,55,4,1,6
1984,148,4,14,8


## Advanced

### Part 6: Sorting

1. Let's start with a query which gets the average price for each type. It will be helpful to alias the average price column as 'avg_price'.

In [27]:
pd.read_sql("""SELECT type,
                      AVG(price) as avg_price
               FROM meals
               GROUP BY type;""",
            conn)

,type,avg_price
0,mexican,9.697595
1,italian,11.292614
2,chinese,9.518717
3,french,11.542000
4,japanese,9.380488
5,vietnamese,9.283019


2. To make it easier to read, sort the results by the `type` column. You can do this with an [ORDER BY](http://www.postgresqltutorial.com/postgresql-order-by/) clause.

In [28]:
pd.read_sql("""SELECT type,
                      AVG(price) as avg_price
               FROM meals
               GROUP BY type
               ORDER BY type;""",
            conn)

,type,avg_price
0,chinese,9.518717
1,french,11.542000
2,italian,11.292614
3,japanese,9.380488
4,mexican,9.697595
5,vietnamese,9.283019


3. Now return the same table again, except this time order by the price in descending order (add the `DESC` keyword).

In [29]:
pd.read_sql("""SELECT type,
                      AVG(price) as avg_price
               FROM meals
               GROUP BY type
               ORDER BY type DESC;""",
            conn)

,type,avg_price
0,vietnamese,9.283019
1,mexican,9.697595
2,japanese,9.380488
3,italian,11.292614
4,french,11.542000
5,chinese,9.518717


3. Sometimes we want to sort by two columns. Write a query to get all the meals, but sort by the type and then by the price. You should have an order by clause that looks something like this: `ORDER BY col1, col2`.

In [30]:
pd.read_sql("""SELECT type,
                      price
               FROM meals
               ORDER BY type, price;""",
            conn)

,type,price
0,chinese,6
1,chinese,6
2,chinese,6
3,chinese,6
4,chinese,6
...,...,...
1988,vietnamese,13
1989,vietnamese,13
1990,vietnamese,13
1991,vietnamese,13


4. For shorthand, people sometimes use numbers to refer to the columns in their order by or group by clauses. The numbers refer to the order they are in the select statement. For instance `SELECT type, dt FROM meals ORDER BY 1;` would order the results by the `type` column.

In [31]:
pd.read_sql("""SELECT type, price   
               FROM meals
               ORDER BY 1, 2;""",
            conn)

,type,price
0,chinese,6
1,chinese,6
2,chinese,6
3,chinese,6
4,chinese,6
...,...,...
1988,vietnamese,13
1989,vietnamese,13
1990,vietnamese,13
1991,vietnamese,13


### Part 7: Joins

Now we are ready to do operations on multiple tables. A [JOIN](https://www.w3schools.com/sql/sql_join.asp) allows us to combine multiple tables by matching on a shared column.

1. Write a query to get one table that joins the `events` table with the `users` table (on `userid`) to create the following result.

    ```
	userid	campaign_id	meal_id	event
0	3	FB	18	bought
1	7	PI	1	like
2	10	TW	29	bought
3	11	RE	19	share
    ...
    ```

In [32]:
pd.read_sql("""SELECT users.userid,
                      users.campaign_id,
                      events.meal_id,
                      events.event
               FROM events
               INNER JOIN users
               ON events.userid = users.userid;""",
           conn)

,userid,campaign_id,meal_id,event
0,3,FB,18,bought
1,7,PI,1,like
2,10,TW,29,bought
3,11,RE,19,share
4,15,RE,33,like
...,...,...,...,...
318115,5513,FB,1992,like
318116,5515,TW,1993,like
318117,5517,RE,1988,like
318118,5519,RE,1990,share


2. Also include information about the meal, like the `type` and the `price`. Only include the `bought` events. The result should look like this:

    ```
  	userid	campaign_id	meal_id	type	price
0	3	FB	18	french	9
1	10	TW	29	italian	15
2	18	TW	40	japanese	13
3	22	RE	23	mexican	12
4	25	FB	8	french	
    ...
    ```

    If your results are different, make sure you filtered it so you only got the `bought` events. You should be able to do this *without* using a where clause, only on clause(s)! (note this has no effect on performance, and may make the query more confusing)

In [33]:
pd.read_sql("""SELECT users.userid,
                      users.campaign_id,
                      events.meal_id,
                      meals.type,
                      meals.price
               FROM events
               INNER JOIN users
               ON events.userid = users.userid
               INNER JOIN meals
               ON events.meal_id = meals.meal_id
               AND events.event = 'bought';""",
           conn)

,userid,campaign_id,meal_id,type,price
0,3,FB,18,french,9
1,10,TW,29,italian,15
2,18,TW,40,japanese,13
3,22,RE,23,mexican,12
4,25,FB,8,french,14
...,...,...,...,...,...
64264,5435,TW,1988,japanese,13
64265,5454,TW,1989,japanese,9
64266,5460,PI,1989,japanese,9
64267,5483,FB,1990,chinese,11


3. Write a query to get how many of each type of meal were bought.

    You should again be able to do this *without* a where clause!

*Phew!* If you've made it this far, congratulations! You're ready to move on to subqueries.

In [34]:
pd.read_sql("""SELECT meals.type,
                      COUNT(*) as total_meals_bought
               FROM events
               INNER JOIN users
               ON events.userid = users.userid
               INNER JOIN meals
               ON events.meal_id = meals.meal_id
               AND events.event = 'bought'
               GROUP BY meals.type;""",
           conn)

,type,total_meals_bought
0,chinese,6267
1,french,16179
2,italian,22575
3,japanese,6921
4,mexican,8792
5,vietnamese,3535


### Part 8: Subqueries

In a [subquery](http://www.postgresql.org/docs/8.1/static/functions-subquery.html), you have a select statement embedded in another select statement.

1. Write a query to get meals that are above the average meal price.

    Start by writing a query to get the average meal price. Then write a query where you put `price > (SELECT ...)` (that select statement should return the average price).

In [35]:
pd.read_sql("""SELECT *
               FROM meals
               WHERE price >
               (SELECT AVG(price)
               FROM meals)""",
            conn)

,meal_id,type,dt,price
0,2,chinese,2013-01-01,13
1,5,chinese,2013-01-03,12
2,8,french,2013-01-03,14
3,9,italian,2013-01-03,13
4,12,mexican,2013-01-03,12
...,...,...,...,...
986,1988,japanese,2013-12-31,13
987,1990,chinese,2013-12-31,11
988,1991,mexican,2013-12-31,11
989,1992,italian,2013-12-31,14


2. Write a query to get the meals that are above the average meal price *for that type*.

    Here you'll need to use a join. First write a query that gets the average meal price for each type. Then join with that table to get ones that are larger than the average price for that meal. Your query should look something like this:

    ```sql
    SELECT meals.*
    FROM meals
    JOIN (SELECT ...) average
    ON ...
    ```

    Note that you need to fill in the select statement that will get the average meal price for each type. We *alias* this table and give it the name `average` (you can include the `AS` keyword, but it doesn't matter).

In [36]:
pd.read_sql("""SELECT *
               FROM meals
               INNER JOIN
               (SELECT type, AVG(price)
               FROM meals
               GROUP BY type) avgs
               ON meals.type = avgs.type
               WHERE meals.price > avgs.avg""",
            conn)

,meal_id,type,dt,price,type,avg
0,2,chinese,2013-01-01,13,chinese,9.518717
1,5,chinese,2013-01-03,12,chinese,9.518717
2,8,french,2013-01-03,14,french,11.542000
3,9,italian,2013-01-03,13,italian,11.292614
4,12,mexican,2013-01-03,12,mexican,9.697595
...,...,...,...,...,...,...
963,1988,japanese,2013-12-31,13,japanese,9.380488
964,1990,chinese,2013-12-31,11,chinese,9.518717
965,1991,mexican,2013-12-31,11,mexican,9.697595
966,1992,italian,2013-12-31,14,italian,11.292614
